In [1]:
# import pandas module
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
import yaml

In [2]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.palettes import Turbo256
from bokeh.models import Span, CustomJS, HoverTool
from bokeh.models.widgets import Select, Panel, Tabs
import panel as pn
from bokeh.layouts import column, layout
from bokeh.transform import dodge
import plotly.graph_objects as go
pn.extension('plotly')

output_notebook()

Loading BokehJS ...

In [3]:
def get_config():
    with open('config.yaml', 'r') as stream:
        config = yaml.safe_load(stream)
    return config

In [4]:
def load_files(subject):
    """
    Function loads the preprocessed files for the given subject; the pre-processing can be found in 'preprocessing_mental_health'
    
    Parameters
        subject:            The subject's initial
    
    Returns
        mental_data:       The dataframe containting the information for the 6 weeeks period
        baseline_data:     The dataframe that is created for the vegan weeks
        vegan_data:        The dataframe that is created for the baseline weeks
    
    """

    # give the path where the needed files are located; with the function get_config
    config = get_config()
    mental_state_files_path = (config['mental_state_files_path'])

    # give the path where the processed files are located
    processed_files_path = mental_state_files_path + '/processed'

    # create the mental file
    mental_filename = processed_files_path + f'/mental_{subject}.csv'

    # create the baseline file
    baseline_filename = processed_files_path + f'/baseline_{subject}.csv'

    # create the vegan file
    vegan_filename = processed_files_path + f'/vegan_{subject}.csv'
    
    # create the mental average file
    mental_avg_filename = processed_files_path + f'/mental_{subject}_mean.csv'

    # create the baseline average file
    baseline_avg_filename = processed_files_path + f'/baseline_{subject}_mean.csv'

    # create the vegan average file
    vegan_avg_filename = processed_files_path + f'/vegan_{subject}_mean.csv'
    
    # create the vegan weekly average file
    mental_weekly_avg_filename = processed_files_path + f'/vegan_{subject}_weekly_mean.csv'
       

    # create dataframes for the whole period (mental), baseline and vegan period
    mental_data = pd.read_csv(mental_filename, sep=',')
    baseline_data = pd.read_csv(baseline_filename, sep=',')
    vegan_data = pd.read_csv(vegan_filename, sep=',')

    mental_data_avg = pd.read_csv(mental_avg_filename, sep=',')
    baseline_data_avg = pd.read_csv(baseline_avg_filename, sep=',')
    vegan_data_avg = pd.read_csv(vegan_avg_filename, sep=',')
    
    mental_weekly_avg = pd.read_csv(mental_weekly_avg_filename, sep=',')
    
    # set 'date' column to datetime 
    mental_data['date'] = pd.to_datetime(mental_data['date'], format='%Y%m%d')
    baseline_data['date'] = pd.to_datetime(baseline_data['date'], format='%Y%m%d')
    vegan_data['date'] = pd.to_datetime(vegan_data['date'], format='%Y%m%d')
    
    return mental_data, baseline_data, vegan_data, mental_data_avg, baseline_data_avg, vegan_data_avg, mental_weekly_avg

In [5]:
def line_plot(subject, mental_data):
    """
    function returns a line plot for the whole 6 weeks including baseline and vegan period
    
    Parameters
        subject:            The subject's initial
        mental_data:        The dataframe with the whole 6 weeks of mental health data
    
    Returns
        line plot for the given subject for over 6 weeks with a separation line at the point where
        the diet was switched.
    
    """
    # give source for line plot
    source = ColumnDataSource(data=mental_data)
    
    # create separate dataframe with only numeric data which is to plot
    numeric_data = mental_data.drop(['current companion', 'current activity', 'location', 'notes', 'subject', 'usage'], axis=1)

    # create empty list where all columns are to be added, set index to 0
    options = []
    index = 0

    # loop through columns and add to list of column names
    for (columnName, columnData) in numeric_data.iteritems():
        if index != 0:
            options.append(columnName)
        index = index + 1 

    # create dropdown menu to select which information top plot
    Axesselect = Select(title="Choose mood:", value="relaxed", options=options)
    Axesselect.js_on_change('value', CustomJS(args=dict(source=source, Axesselect=Axesselect), code="""
        source.data['relaxed'] = source.data[Axesselect.value];
        source.change.emit();
    """))

    controls = [Axesselect]
    inputs = column(*controls, width=300)

    # Create a plot
    p = figure(title=f'Subject {subject}:', plot_width=1000, plot_height=400, y_range=(0,7) , 
                x_axis_type='datetime', x_axis_label='Date', y_axis_label='Mood score')
    p.line('date', 'relaxed', source=source, color='#85D0FF')

    # add tooltips to the plot to show the exact date and scores; as well as the textual data which cannot be plotted to
    # provive additional information.
    tooltips = HoverTool(
        tooltips = [("date", "@date{%F}"),
                    ('Score', '$y'),
                    ('Notes', '@notes'),
                    ('Current companion', '@{current companion}'),
                    ('Usage', '@{usage}'),
                    ('Location', '@location')],
        formatters={'@date': 'datetime'}
    )
    
    # add the tooltips to the plot
    p.add_tools(tooltips)
    
    # set line at the border of baseline/vegan period
    border = time.mktime(datetime(2020, 10, 26, 3, 0, 0).timetuple())*1000
    daylight_savings_start = Span(location=border, dimension='height', line_color='#AAFF99', line_width=3)
    p.add_layout(daylight_savings_start)

    # finish off by formatting the dropdown and plot
    final_layout = layout([[inputs, p]])
    show(final_layout)

In [6]:
def bar_plot(baseline_data_avg, vegan_data_avg):
    """
    function returns a line plot for the whole 6 weeks including baseline and vegan period
    
    Parameters
        subject:            The subject's initial
    
    Returns
        line plot for the given subject for over 6 weeks with a separation line at the point where
        the diet was switched.
    
    """
    vegan_baseline = pd.merge(baseline_data_avg, vegan_data_avg, on='mood')
    vegan_baseline = vegan_baseline.rename(columns={'score_x': 'baseline', 'score_y': 'vegan'})

    moods = vegan_baseline['mood'].unique().tolist()

    source = ColumnDataSource(vegan_baseline)

    p = figure(x_range=moods, y_range=(0, 7), title=f"Average moods baseline vs vegan for subject {subject}", plot_width=900)

    p.vbar(x=dodge('mood', -0.2, range=p.x_range), top='baseline', width=0.4, source=source, color="#8AE1FC", legend_label='Baseline')

    p.vbar(x=dodge('mood',  0.2,  range=p.x_range), top='vegan', width=0.4, source=source, color="#C3EB78", legend_label='Vegan')
    
    p.add_tools(HoverTool(
        tooltips = [('Vegan', '@vegan'),
                   ('Baseline', '@baseline')]
    ))

    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.legend.location = "top_left"
    p.xaxis.major_label_orientation = "vertical"
    p.legend.click_policy= 'hide'
    p.legend.orientation = "horizontal"

    show(p)

In [7]:
def radar_plot(subject, baseline_data_avg, vegan_data_avg):
    """
    function returns a radar plot comparing vegan and baseline

    Returns
        Radar plot with average scores during the baseline and the vegan period
    """

    moods = baseline_data_avg['mood'].unique().tolist()
    
    p = go.Figure()

    p.add_trace(go.Scatterpolar(
        type='scatterpolar',
        r=baseline_data_avg['score'],
        theta = moods,
        fill='toself',
        line_color = '#60D7FB',
        fillcolor = '#8AE1FC',
        opacity = 0.5,
        name='Baseline'
    ))
    p.add_trace(go.Scatterpolar(
        r=vegan_data_avg['score'],
        theta = moods,
        fill='toself',
        line_color = '#B8E75F',
        fillcolor = '#C3EB78',
        opacity = 0.5,
        name='Vegan'

    ))

    p.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
          range=[0, 7]
        )),
        showlegend=True,
        title = f'Total average mood scores of baseline vs. vegan for subject {subject}'
    )

    return p

In [8]:
def radar_plot_weekly(subject, mental_weekly_avg):
    """
    function returns a radar plot per week showing the weekly averages
    
    
    Returns
        Radar plot with average scores during the baseline and the vegan period; shown per week
    """
   
    moods = mental_weekly_avg['mood'].unique().tolist()

    tabs = pn.Tabs()

    columns = iter(mental_weekly_avg)
    next(columns)

    for column in columns:
        p = go.Figure(data=go.Scatterpolar(
            r=mental_weekly_avg[column],
            theta=moods,
            fill='toself',
            line_color = '#60D7FB',
            fillcolor = '#8AE1FC',
            opacity = 0.5
        ))


        p.update_layout(
          polar=dict(
            radialaxis=dict(
              visible=True,
                range=[0,7]
            ),
          ),
            showlegend=False,
            title = f'Average mood scores of {column} for subject {subject}'
        )

        tabs.append((column, p))

    return tabs

In [9]:
def combine_radar():
    """
    function combines radar_plot function and radar_weekly_plot 
    
    
    Returns
        Returns a tabular figure where to choose between average or weekly
    """
   
    weekly = radar_plot_weekly(subject, mental_weekly_avg)

    average = radar_plot(subject, baseline_data_avg, vegan_data_avg)
    
    tabs = pn.Tabs(('Weekly average', weekly), ('Total average', average))

    return tabs

In [16]:
subject = 'd'
mental_data = load_files(subject)[0]
baseline_data = load_files(subject)[1]
vegan_data = load_files(subject)[2]

mental_data_avg = load_files(subject)[3]
baseline_data_avg = load_files(subject)[4]
vegan_data_avg = load_files(subject)[5]

mental_weekly_avg = load_files(subject)[6]

In [19]:
combine_radar()

Tabs
    [0] Tabs
        [0] Plotly(Figure, name='baseline week 1')
        [1] Plotly(Figure, name='baseline week 2')
        [2] Plotly(Figure, name='vegan week 1')
        [3] Plotly(Figure, name='vegan week 2')
        [4] Plotly(Figure, name='vegan week 3')
        [5] Plotly(Figure, name='vegan week 4')
    [1] Plotly(Figure, name='Total average')

In [ ]:
def mental_total_averages(baseline_data, vegan_data):
    """
    Author: Henrike Vaartstra
     
    function creates a new dataframe with the total averages of baseline and vegan period combined
    
    Parameters
        baseline_data:            The baseline data which is loaded in the function load_mental_files
        vegan_data:               The vegan data which is loaded in the function load_mental_files
    
    Returns
        A dataframe called vegan_baseline containing the average mood scores for the baseline and 
        vegan period in two separate columns.
    
    """

    baseline_mean = baseline_data.drop('date', axis=1).mean().round(2).dropna()
    vegan_mean = vegan_data.drop('date', axis=1).mean().round(2).dropna()
    
    baseline_mean = baseline_mean.to_frame().reset_index()
    vegan_mean = vegan_mean.to_frame().reset_index()

    vegan_baseline = pd.merge(vegan_mean, baseline_mean, on='index')
    vegan_baseline = vegan_baseline.rename(columns={'0_x': 'vegan', '0_y': 'baseline', 'index':'mood'})
    
    return vegan_baseline

def mental_weekly_averages(baseline_data, vegan_data):
    """
    Author: Henrike Vaartstra
     
    function creates a new dataframe with the weekly averages of baseline and vegan period combined
    
    Parameters
        baseline_data:            The baseline data which is loaded in the function load_mental_files
        vegan_data:               The vegan data which is loaded in the function load_mental_files
    
    Returns
        A dataframe called vegan_baseline containing the weekly average mood scores for the baseline and 
        vegan period in 6 columns; one for each week.
    
    """
    baseline_w_mean = baseline_data.set_index('date').resample('w').mean().round(2).reset_index().drop('date', axis=1).T.reset_index()
    vegan_w_mean = vegan_data.set_index('date').resample('w').mean().round(2).reset_index().drop('date', axis=1).T.reset_index()

    vegan_baseline_w = pd.merge(vegan_w_mean, baseline_w_mean, on='index')
    vegan_baseline_w = vegan_baseline_w.rename(columns={'0_x': 'vegan week 1', '1_x': 'vegan week 2', 2: 'vegan week 3', 
                                                3: 'vegan week 4',  '0_y': 'baseline week 1',
                                                '1_y': 'baseline week 2', 'index':'mood'})
    vegan_baseline_w = vegan_baseline_w[['mood', 'baseline week 1', 'baseline week 2', 
                                 'vegan week 1', 'vegan week 2', 'vegan week 3', 'vegan week 4']]
    
    return vegan_baseline_w